In [1]:
from pptx import Presentation
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import spacy
from nltk.corpus import conll2000
from nltk.corpus import conll2002
import re # Regular expression operations
nltk.download('conll2002')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('conll2000')
nltk.download('treebank')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('ieer')

[nltk_data] Downloading package conll2002 to
[nltk_data]     /Users/zeyutan/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!
[nltk_data] Downloading package punkt to /Users/zeyutan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/zeyutan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     /Users/zeyutan/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     /Users/zeyutan/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/zeyutan/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/zeyutan/nltk_data...
[nltk_data]   Package wo

True

In [13]:
'''
Step 1: get the raw text from pptx files
'''

f= open("text_extracted_with_newline.txt","w+")
f_raw = open("raw_text.txt",'w+')
success = 0
fail = 0
test_text = ''
sentences = []
with os.scandir('./output') as it:
    for entry in it:
        if entry.is_file():
            print(entry.name, entry.path)
            try:
                cur_text = ''
                prs = Presentation(entry.path)
                for slide in prs.slides:
                    for shape in slide.shapes:
                        if hasattr(shape, "text"):
                            sentences.extend(shape.text.split('. '))
                            f_raw.write(shape.text)
                success += 1
            except Exception:
                fail += 1
                pass
        break
while("" in sentences) :
    sentences.remove("")

nonempty_sentences = []
for sentence in sentences:
    single_str_list = sentence.split(' ')
    if len(single_str_list) > 1:
        nonempty_sentences.append(sentence)

cur_sen = 0
# print(len(nonempty_sentences))
for sentence in nonempty_sentences:
    cur_sen += 1
#         sentence.replace('\n',' ')
    if cur_sen == len(nonempty_sentences):
        sentence = sentence
    else:
        sentence = sentence +'\n'  
    f.write(sentence)

f.close()

line_num = 0
f = open("text_extracted_without_newline.txt","w+")
with open('text_extracted_with_newline.txt','r') as file:
    for line in file:
        line_num += 1
        if line != '\n':
            f.write(line)
f.close()
file.close()


f7e0fd2251740b9f10436eaa2b92a7d8.pptx ./output/f7e0fd2251740b9f10436eaa2b92a7d8.pptx


In [37]:
'''
step 2 method collection
'''

def ie_preprocess(raw_text):
    sentences = nltk.sent_tokenize(raw_text)
#     print("segmentation result: ")
#     print(sentences)
#     print("-----------------------------------------------------")

    sentences = [nltk.word_tokenize(word) for word in sentences]
#     print("tokennization result: ")
#     print_list_of_list(sentences)
#     print("-----------------------------------------------------")

    sentences = [nltk.pos_tag(word) for word in sentences]
#     print("part of speach tagging result: ")
#     print_list_of_list(sentences)
    
    return sentences
    
def print_list(l):
    for element in l:
        print(element)

def print_list_of_list(ll):
    for l in ll:
        print_list(l)

In [30]:
'''
1, 1.1
Step 2: segmentize(seperate str in to sentences),tokenize (seperate sentences into words), 
pos tag(part-of-speech tag) the raw text
'''

doc_test1 = "Today is a fine day in Madison. Mr.Tan would like to take a walk."
sentences = ie_preprocess(doc_test1)



segmentation result: 
['Today is a fine day in Madison.', 'Mr.Tan would like to take a walk.']
-----------------------------------------------------
tokennization result: 
Today
is
a
fine
day
in
Madison
.
Mr.Tan
would
like
to
take
a
walk
.
-----------------------------------------------------
part of speach tagging result: 
('Today', 'NN')
('is', 'VBZ')
('a', 'DT')
('fine', 'JJ')
('day', 'NN')
('in', 'IN')
('Madison', 'NNP')
('.', '.')
('Mr.Tan', 'NNP')
('would', 'MD')
('like', 'VB')
('to', 'TO')
('take', 'VB')
('a', 'DT')
('walk', 'NN')
('.', '.')


In [5]:
'''
2, 2.1
find Noun phrase chunks
create grammar -> use grammar to create regular expression parser -> 
use parser parse sentence with part of speach tagging
'''

#define a simple grammar with a single regular-expression pattern rule
grammar = "NP: {<DT>?<JJ>*<NN>}" # an optional determiner + any number of adj + a noun

#create a chunk parser
cp = nltk.RegexpParser(grammar)

#parse the sentence
result = cp.parse(sentences[0])
print(result)

#result.draw() #visually represent the sentence with noun phrase chunked

(S
  (NP Today/NN)
  is/VBZ
  (NP a/DT fine/JJ day/NN)
  in/IN
  Madison/NNP
  ./.)


In [12]:
'''
2.2
the grammar doesn't have to be regular expression pattern, it can also be tag pattern
A tag pattern is a sequence of part-of-speech tags delimited using angle brackets, 
e.g. <DT>?<JJ>*<NN>. | <DT>?<JJ.*>*<NN.*>+.
'''
sentence_test1 = [("the", "DT"), ("little", "JJ"), ("yellow", "JJ"),
                  ("dog", "NN"), ("barked", "VBD"), ("at", "IN"),  ("the", "DT"), ("cat", "NN")]
#define a simple grammar with a tag pattern rule
grammar = "NP: {<DT>?<JJ.*>*<NN.*>+}" # any sequence of tokens beginning with an optional determiner, 
                                      # followed by zero or more adjectives of any type (including relative adjectives like earlier/JJR), 
                                      # followed by one or more nouns of any type.

#create a chunk parser
cp = nltk.RegexpParser(grammar)

#parse the sentence
result = cp.parse(sentence_test1)
print(result)
result = cp.parse(sentences[0])
print(result)

(S
  (NP the/DT little/JJ yellow/JJ dog/NN)
  barked/VBD
  at/IN
  (NP the/DT cat/NN))
(S
  (NP Today/NN)
  is/VBZ
  (NP a/DT fine/JJ day/NN)
  in/IN
  (NP Madison/NNP)
  ./.)


In [ ]:
'''
2.3 multiple regular expression rules
The chunking rules are applied in turn, successively updating the chunk structure.
return result when all rules are applied
'''

In [15]:
'''
2.5 use chink to exclude tag: }...{
'''
grammar = r"""
  NP:
    {<.*>+}          # Chunk everything
    }<VBD|IN>+{      # Chink sequences of VBD and IN
  """
sentence = [("the", "DT"), ("little", "JJ"), ("yellow", "JJ"),
       ("dog", "NN"), ("barked", "VBD"), ("at", "IN"),  ("the", "DT"), ("cat", "NN")]
cp = nltk.RegexpParser(grammar)
print(cp.parse(sentence))

(S
  (NP the/DT little/JJ yellow/JJ dog/NN)
  barked/VBD
  (NP at/IN the/DT cat/NN))


In [ ]:
'''
2.6
represent chunk
1. IOB(I:inside the chunk O: outside the chunk B: beginning of the chunk)
2. tree
'''

In [7]:
'''
3, 3.1
Conll2000 text corpus       
'''
print(conll2000.chunked_sents('train.txt')[99])# conll2000 has tain and test set in pos tag + IOB form

print(conll2000.chunked_sents('train.txt', chunk_types=['NP'])[99]) # use chunk parameter to only see np

(S
  (PP Over/IN)
  (NP a/DT cup/NN)
  (PP of/IN)
  (NP coffee/NN)
  ,/,
  (NP Mr./NNP Stone/NNP)
  (VP told/VBD)
  (NP his/PRP$ story/NN)
  ./.)
(S
  Over/IN
  (NP a/DT cup/NN)
  of/IN
  (NP coffee/NN)
  ,/,
  (NP Mr./NNP Stone/NNP)
  told/VBD
  (NP his/PRP$ story/NN)
  ./.)


In [41]:
class UnigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)] for sent in train_sents]# map each chunk tree to 
                                                                                                   # a list of word,tag,chunk triples
        self.tagger = nltk.UnigramTagger(train_data) # use training data to train a unigram tagger, save the tagger as self.tagger

    def parse(self, sentence): # sentence = tagged sentence
        # extracting the part-of-speech tags from that sentence
        pos_tags = [pos for (word,pos) in sentence] 
        
        # tags the part-of-speech tags with IOB chunk tags, trained in constructor
        tagged_pos_tags = self.tagger.tag(pos_tags) 
        
        # extracts the chunk tags, and combines them with the original sentence, to yield conlltags. 
        # unzip the sentence to(word,pos)turple
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags] 
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        
        # uses conlltags2tree to convert the result back into a chunk tree
        return nltk.chunk.conlltags2tree(conlltags)

    
    
class BigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.BigramTagger(train_data)

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)    
    
    
    

In [33]:
'''
3.2
Simple Evaluation and Baselines
'''
# make the trivial chunk parser cp that creates no chunks the baseline
cp = nltk.RegexpParser("")
test_sents = conll2000.chunked_sents('test.txt', chunk_types=['NP'])
print(cp.evaluate(test_sents))# 43.4% of words are in the chunk(NP chunk, because chunk_types = ["NP"])
print("----------------------------")

grammar = r"NP: {<[CDJNP].*>+}"
test_sents = conll2000.chunked_sents('test.txt', chunk_types=['NP'])
cp = nltk.RegexpParser(grammar)
print(cp.evaluate(test_sents))
print("----------------------------")

'''unigram tagger is a tagger that only uses a single word as its context for determining the POS(Part-of-Speech) tag.
In simple words, Unigram Tagger is a context-based tagger whose context is a single word, i.e., Unigram.'''


'''use the training corpus to find the chunk tag (I, O, or B) that is most likely for each part-of-speech tag.
e.g determine the correct chunk tag, given each word's part-of-speech tag, not word itself. (j -> B,I)

want to create a unigram tagger -> a list of training sentences, which will be in the form of chunk trees ->
use nltk.chunk.tree2conlltags(sent) to convert tree to IOB form(map each chunk tree to a list of word,tag,chunk triples) ->

'''
# train UnigramChunker using the CoNLL 2000 corpus
test_sents = conll2000.chunked_sents('test.txt', chunk_types=['NP'])
train_sents = conll2000.chunked_sents('train.txt', chunk_types=['NP'])
unigram_chunker = UnigramChunker(train_sents)
print(sentences)
print(unigram_chunker.parse(sentences[0]))

print(unigram_chunker.evaluate(test_sents))
print("----------------------------")

# each type pos tag represents chunk tag
postags = sorted(set(pos for sent in train_sents for (word,pos) in sent.leaves()))
print(unigram_chunker.tagger.tag(postags))
print("----------------------------")

# use Bigramchunker
bigram_chunker = BigramChunker(train_sents)
print(bigram_chunker.evaluate(test_sents))

ChunkParse score:
    IOB Accuracy:  43.4%%
    Precision:      0.0%%
    Recall:         0.0%%
    F-Measure:      0.0%%
----------------------------
ChunkParse score:
    IOB Accuracy:  87.7%%
    Precision:     70.6%%
    Recall:        67.8%%
    F-Measure:     69.2%%
----------------------------
[[('Today', 'NN'), ('is', 'VBZ'), ('a', 'DT'), ('fine', 'JJ'), ('day', 'NN'), ('in', 'IN'), ('Madison', 'NNP'), ('.', '.')], [('Mr.Tan', 'NNP'), ('would', 'MD'), ('like', 'VB'), ('to', 'TO'), ('take', 'VB'), ('a', 'DT'), ('walk', 'NN'), ('.', '.')]]
(S
  (NP Today/NN)
  is/VBZ
  (NP a/DT fine/JJ day/NN)
  in/IN
  (NP Madison/NNP)
  ./.)
ChunkParse score:
    IOB Accuracy:  92.9%%
    Precision:     79.9%%
    Recall:        86.8%%
    F-Measure:     83.2%%
----------------------------
[('#', 'B-NP'), ('$', 'B-NP'), ("''", 'O'), ('(', 'O'), (')', 'O'), (',', 'O'), ('.', 'O'), (':', 'O'), ('CC', 'O'), ('CD', 'I-NP'), ('DT', 'B-NP'), ('EX', 'B-NP'), ('FW', 'I-NP'), ('IN', 'O'), ('JJ', 'I-NP')

In [ ]:
'''
3.3
1. regular-expression based chunkers
2. n-gram chunkers(Bigram or Unigram chunker)
1 and 2 chunkers decide what chunks to create entirely based on part-of-speech tags

Some sentence we have exactly the same pos tags but are chunked differently:
    1> Joey/NN sold/VBD the/DT farmer/NN rice/NN ./.
    2> Nick/NN broke/VBD my/DT computer/NN monitor/NN ./.
    
So pos tag is not sufficient and 1,2 chunkers cannot be used
we need to make use of information about the content of the words, 
in addition to just their part-of-speech tags, if we wish to maximize chunking performance.
Classifier-Based Chunkers should be used
'''

In [43]:
'''
4.1
chunk structures have been relatively flat, 1 layer(NP,VP,PP).
It is possible to build chunk structures of arbitrary depth, 
simply by creating a multi-stage chunk grammar containing recursive rules. 
'''

# a four-stage chunk grammar, and can be used to create structures having a depth of at most four.
grammar = r"""
  NP: {<DT|JJ|NN.*>+}          # Chunk sequences of DT, JJ, NN
  PP: {<IN><NP>}               # Chunk prepositions followed by NP
  VP: {<VB.*><NP|PP|CLAUSE>+$} # Chunk verbs and their arguments
  CLAUSE: {<NP><VP>}           # Chunk NP, VP
  """
cp = nltk.RegexpParser(grammar)
sentence = [("Mary", "NN"), ("saw", "VBD"), ("the", "DT"), ("cat", "NN"),
    ("sit", "VB"), ("on", "IN"), ("the", "DT"), ("mat", "NN")]

print(cp.parse(sentence)) # sit on the mat (vp(pp(np))), however, it missed a vp headed by "saw"

# add an optional second argument loop to specify the number of times the set of patterns should be run
# more clauses can be found 
cp = nltk.RegexpParser(grammar, loop = 2) 
sentence = [("John", "NNP"), ("thinks", "VBZ"), ("Mary", "NN"),
     ("saw", "VBD"), ("the", "DT"), ("cat", "NN"), ("sit", "VB"),
     ("on", "IN"), ("the", "DT"), ("mat", "NN")]
print(cp.parse(sentence))

(S
  (NP Mary/NN)
  saw/VBD
  (CLAUSE
    (NP the/DT cat/NN)
    (VP sit/VB (PP on/IN (NP the/DT mat/NN)))))
(S
  (NP John/NNP)
  thinks/VBZ
  (CLAUSE
    (NP Mary/NN)
    (VP
      saw/VBD
      (CLAUSE
        (NP the/DT cat/NN)
        (VP sit/VB (PP on/IN (NP the/DT mat/NN)))))))


In [60]:
'''
4.2
A tree is a set of connected labeled nodes, each reachable by a unique path from a distinguished root node. 
Create trees in nltk
'''
#In NLTK, we create a tree by giving a node label and a list of children
tree1 = nltk.Tree('NP', ['Alice'])
print(tree1)
print("--------------------------")

tree2 = nltk.Tree('NP', ['the', 'rabbit'])
print(tree2)
print("--------------------------")

# incorporate tree 1 and tree 2 into successively larger trees
tree3 = nltk.Tree('VP', ['chased', tree2])
tree4 = nltk.Tree('S', [tree1, tree3])
print(tree4)
print("--------------------------")

print(tree4[1].label())
print("--------------------------")

print(tree4.leaves())
print("--------------------------")

print(tree4[1][1][1]) # s -> vp -> np -> "rabbit"

# tree4.draw()  # visualize the tree

(NP Alice)
--------------------------
(NP the rabbit)
--------------------------
(S (NP Alice) (VP chased (NP the rabbit)))
--------------------------
VP
--------------------------
['Alice', 'chased', 'the', 'rabbit']
--------------------------
rabbit


In [71]:
'''
Step 3: entity detection
5 Named entity recognition

named entities (NEs), definite noun phrases that refer to specific types of individuals,
such as organizations, persons, dates, and so on.

Goal of a named entity recognition (NER) system is to identify all textual mentions of the named entities.
This can be broken down into two sub-tasks: 
1. identifying the boundaries of the NE 
2. identifying NE type.

'''
# NLTK provides a classifier that has already been trained to recognize named entities, 
# accessed with the function nltk.ne_chunk().
sent = nltk.corpus.treebank.tagged_sents()[22] # list of turple of words with pos tag
print(nltk.ne_chunk(sent, binary=True))

# If we set the parameter binary=True, then named entities are just tagged as NE;
# otherwise, the classifier adds category labels such as PERSON, ORGANIZATION, and GPE
print(nltk.ne_chunk(sent)) 

(S
  The/DT
  (NE U.S./NNP)
  is/VBZ
  one/CD
  of/IN
  the/DT
  few/JJ
  industrialized/VBN
  nations/NNS
  that/WDT
  *T*-7/-NONE-
  does/VBZ
  n't/RB
  have/VB
  a/DT
  higher/JJR
  standard/NN
  of/IN
  regulation/NN
  for/IN
  the/DT
  smooth/JJ
  ,/,
  needle-like/JJ
  fibers/NNS
  such/JJ
  as/IN
  crocidolite/NN
  that/WDT
  *T*-1/-NONE-
  are/VBP
  classified/VBN
  *-5/-NONE-
  as/IN
  amphobiles/NNS
  ,/,
  according/VBG
  to/TO
  (NE Brooke/NNP)
  T./NNP
  Mossman/NNP
  ,/,
  a/DT
  professor/NN
  of/IN
  pathlogy/NN
  at/IN
  the/DT
  (NE University/NNP)
  of/IN
  (NE Vermont/NNP College/NNP)
  of/IN
  (NE Medicine/NNP)
  ./.)
(S
  The/DT
  (GPE U.S./NNP)
  is/VBZ
  one/CD
  of/IN
  the/DT
  few/JJ
  industrialized/VBN
  nations/NNS
  that/WDT
  *T*-7/-NONE-
  does/VBZ
  n't/RB
  have/VB
  a/DT
  higher/JJR
  standard/NN
  of/IN
  regulation/NN
  for/IN
  the/DT
  smooth/JJ
  ,/,
  needle-like/JJ
  fibers/NNS
  such/JJ
  as/IN
  crocidolite/NN
  that/WDT
  *T*-1/-NONE-
  ar

In [84]:
'''
Step 4: relation detection
After NEs are recognized, we want to get the relations between NEs
we will typically be looking for relations between specified types of named entity.

Different method do identifies the turples we want:
1. find (a, x, b), where X and Y are named entities ofthe required types, 
and x is the string of words that intervenes between X and Y. e.g a = PERSON, b = GPE;
then use regular expressions to pull out just those instances of α that express the relation that we are looking for.
    
2. devise patterns that are sensitive to part of speach tags for higher accuracy
(some corpus not only provide annotated NE but also provide pos rags)
'''
# Method 1:
IN = re.compile(r'.*\bin\b(?!\b.+ing)') # search for x = string that contains in and b is not a gerund(动名词)
for doc in nltk.corpus.ieer.parsed_docs('NYT_19980315'):
    for rel in nltk.sem.extract_rels('ORG', 'LOC', doc, corpus='ieer', pattern = IN):
        print(nltk.sem.rtuple(rel))
print("-----------------------------------------------")

# method 2:
vnv = """
(
is/V|    # 3rd sing present and
was/V|   # past forms of the verb zijn ('be')
werd/V|  # and also present
wordt/V  # past of worden ('become)
)
.*       # followed by anything
van/Prep # followed by van ('of')
"""
VAN = re.compile(vnv, re.VERBOSE) # compile the regular expression
for doc in conll2002.chunked_sents('ned.train'):
    for rel in nltk.sem.extract_rels('PER', 'ORG', doc, corpus='conll2002', pattern=VAN):
        print(nltk.sem.clause(rel, relsym="VAN"))

[ORG: 'WHYY'] 'in' [LOC: 'Philadelphia']
[ORG: 'McGlashan &AMP; Sarrail'] 'firm in' [LOC: 'San Mateo']
[ORG: 'Freedom Forum'] 'in' [LOC: 'Arlington']
[ORG: 'Brookings Institution'] ', the research group in' [LOC: 'Washington']
[ORG: 'Idealab'] ', a self-described business incubator based in' [LOC: 'Los Angeles']
[ORG: 'Open Text'] ', based in' [LOC: 'Waterloo']
[ORG: 'WGBH'] 'in' [LOC: 'Boston']
[ORG: 'Bastille Opera'] 'in' [LOC: 'Paris']
[ORG: 'Omnicom'] 'in' [LOC: 'New York']
[ORG: 'DDB Needham'] 'in' [LOC: 'New York']
[ORG: 'Kaplan Thaler Group'] 'in' [LOC: 'New York']
[ORG: 'BBDO South'] 'in' [LOC: 'Atlanta']
[ORG: 'Georgia-Pacific'] 'in' [LOC: 'Atlanta']
-----------------------------------------------
VAN("cornet_d'elzius", 'buitenlandse_handel')
VAN('johan_rottiers', 'kardinaal_van_roey_instituut')
VAN('annie_lennox', 'eurythmics')


In [65]:
with open('raw_text.txt','r') as file:
    ttl_noun_phrase = []
    for line in file:
        #get the raw text from a file
        
        # segmentize, tokenize, and pos tag the token
        sentences = ie_preprocess(line)
        train_sents = conll2000.chunked_sents('train.txt', chunk_types=['NP'])
        unigram_chunker = UnigramChunker(train_sents)
        for sentence in sentences:
            parsed_cur_sentence = unigram_chunker.parse(sentence)
            noun_phrases_cur = [' '.join(leaf[0] for leaf in tree.leaves()) 
                      for tree in parsed_cur_sentence.subtrees() 
                      if tree.label()=='NP'] 
            ttl_noun_phrase.extend(noun_phrases_cur)         
    print("noun phrase: ")    
    print(ttl_noun_phrase)
    word_count = {}
    for noun in ttl_noun_phrase:
        if noun in word_count:
            word_count[noun] += 1
        else:
            word_count[noun] = 1
    
    print("-------------------------------------------")
    print("Word frequency rank:")
    
    word_frequency = list(sorted(word_count.items(), key=lambda item: item[1],reverse = True))
    for rank in word_frequency:
        print(rank)
        

noun phrase: 
['ABFigure S1', 'GC-MS chromatograms', 'ethanol extracts', 'buttons', 'A', 'roots', 'B', 'L. williamsii', 'Figure S2', 'Gene', 'classification', 'the L.', 'transcriptome', 'Unigenes', 'BLASTx matches', 'the Arabidopsis proteins', 'three main GO categories', 'cellular components', 'molecular functions', 'biological processes', 'The left-hand scale', 'the y-axis', 'the percentage', 'unigenes', 'each category', 'The right-hand log scale', 'the y-axis', 'the number', 'unigenes', 'the same category', 'Figure S3', 'The carbon fixation pathway', 'photosynthetic organisms', 'KEGG', 'http', '//www.genome.ad.jp/kegg/', 'The enzymes', 'bold font', 'the EC numbers', 'red', 'those enzymes', 'whose genes', 'the peyote unigene dataset', 'blue', 'enzymes absent', 'Figure S4', 'Glycolysis / Gluconeogenesis biosynthesis', 'the de novo', 'annotation', 'the L.', 'transcriptome', 'This pathway', 'the KEGG Mapper Reconstruct pathway tool', 'http', '//www.genome.ad.jp/kegg/', 'Enzymes', 'the L.